### Defining the A, B, C and D matrices in order to obtain the transfer function H (state-space representation)

Writing the system of second order differential equations of the analysed system. The dimension of the system depends on the number of degress of freedom  

$$
M \ddot{\underline{x}} + C \dot{\underline{x}} + K \underline{x} = \underline{F}
$$

Converting a system of $n$ second order differential equations to a system of 2$n$ first order differential equations

$$
\begin{cases} M \dot{\underline{v}}  + C \underline{v} + K \underline{x} = \underline{F} \\ \dot{\underline{x}}=\underline{v} \end{cases} 
$$

Rewriting the system in the state-space form:

$$
\begin{cases} \dot{\underline{v}} = - M^{-1} C \underline{v} - M^{-1} K \underline{x} + M^{-1}\underline{F} \\ \dot{\underline{x}} = \underline{v} \end{cases} 
$$

$$
 \begin{bmatrix} \dot{\underline{x}} \\ \dot{\underline{v}} \end{bmatrix} = \begin{bmatrix} \underline{\underline{0}} & \underline{\underline{I}} \\-M^{-1}K & -M^{-1}C \end{bmatrix}  \begin{bmatrix} \underline{x} \\ \underline{v} \end{bmatrix} +  \begin{bmatrix} \underline{\underline{0}} \\ M^{-1} \end{bmatrix} \underline{F} 
$$

Having chosen the position of the masses as the output $y$
$$
\underline{y} = \underline{x} =  \begin{bmatrix} \underline{\underline{I}} & \underline{\underline{0}} \end{bmatrix}  \begin{bmatrix} \underline{x} \\ \underline{v} \end{bmatrix} + [\underline{\underline{0}}] \underline{F}
$$

In [1]:
#Given M, C and K

n=size(M,1)

# Define the system matrices
A = spzeros(2n, 2n)
B = spzeros(2n, n)
CC = spzeros(n, 2n)


#First quarter stays as it is

#Second quarter

A[1:n,n+1:2n]=Matrix(1I,n,n)

#Third quarter of the A matrix (Stiffness)

A[n+1:2n,1:n]=-M\K

#Fourth quarter of the A matrix (Damping)

A[n+1:2n,n+1:2n]=-M\C

dropzeros!(A)

#B matrix

B[n+1:2n,1:n]=inv(M)

dropzeros!(B)

#CC matrix

CC[1:n, 1:n]=Matrix(1I,n,n)

dropzeros!(CC)

#Transfer Function

H = ss(A,B,CC,0)


LoadError: UndefVarError: M not defined

### Finding and saving the eigenvalues and eigenvectors of every transfer function corresponding to a specific frequency $\omega$ so it is possible to compare them with Bode diagrams   

In [ ]:
#N-DIMENSION
#insert N

N=size(M,1)

using LinearMaps
using IterativeSolvers
using LaTeXStrings

#studying system amplification through eigenvalues of the transfer matrix

w = LinRange(0.01, 100, 10000)  #frequency range
lamb = spzeros(length(w))
eigvectors = rand(ComplexF64,length(w),N)

for j in 1:length(w)
    om = w[j]
    G = -om^(2)*M + im*C*om + K  #creating the inverse transfer function related to this frequency
    F = lu(G)
    Fmap = LinearMap{ComplexF64}((y, x) -> ldiv!(y, F, x), N, ismutating = true)
    lambda, x = invpowm(Fmap, shift = 0, tol = 1e-8, maxiter = 1000)  #finding minimum eig in absolute value
    lambda = abs(lambda);
    eigvectors[j,:] = x
    lambda = 1/lambda  #finding eig of the transfer function
    lamb[j] = lambda  #saving the value
end

### Overlapping all Bode diagrams and comparing them with the maximum eigenvalue of H for each $\omega$

In [ ]:
#plots

plot(w,lamb,label=L"$\lambda_{max}$", linewidth=1.5, linecolor=:red, xaxis=:log, yaxis=:log)


for i in 1:size(H,1)
    for j in 1:size(H,2)
        bodeplot!(H[i,j], linewidth=0.2, linecolor=:green, plotphase=false, label="", ylim=(10e-10,10e5))
    end
end


#xlabel=L"\omega",ylabel="System Amplification",

savefig("bode_diagram.pdf")